In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv('data.csv')
print(df)

       quality  bits  intra_parts  skip_parts  inter_16x16_parts  \
0           22  2143            0           0                  1   
1           22  1735            0           2                  0   
2           22  1971            0           0                  0   
3           22  2473            0           0                  0   
4           22  1093            0           0                  1   
...        ...   ...          ...         ...                ...   
15995       37    66           16           1                  1   
15996       37   100           32           0                  0   
15997       37   141           16           1                  0   
15998       37   335           16           0                  2   
15999       37   226           32           0                  0   

       inter_4x4_parts  inter_other_parts  non_zero_pixels  frame_width  \
0                    3                 21              404          416   
1                    3           